In [2]:
import pandas as pd

df = pd.read_csv(r"C:\Users\kunal\Downloads\Food_Delivery_Time_Prediction.csv")
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Order_ID                    200 non-null    object 
 1   Customer_Location           200 non-null    object 
 2   Restaurant_Location         200 non-null    object 
 3   Distance                    200 non-null    float64
 4   Weather_Conditions          200 non-null    object 
 5   Traffic_Conditions          200 non-null    object 
 6   Delivery_Person_Experience  200 non-null    int64  
 7   Order_Priority              200 non-null    object 
 8   Order_Time                  200 non-null    object 
 9   Vehicle_Type                200 non-null    object 
 10  Restaurant_Rating           200 non-null    float64
 11  Customer_Rating             200 non-null    float64
 12  Delivery_Time               200 non-null    float64
 13  Order_Cost                  200 non

In [ ]:
# Check missing data
print(df.isnull().sum())

# Fill numerical missing with mean
for col in ['distance', 'temperature', 'humidity']:
    df[col].fillna(df[col].mean(), inplace=True)

# Fill categorical missing with mode
for col in ['weather', 'traffic_conditions']:
    df[col].fillna(df[col].mode()[0], inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder

le_weather = LabelEncoder()
df['weather_encoded'] = le_weather.fit_transform(df['weather'])

df = pd.get_dummies(df, columns=['traffic_conditions'])


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numeric_cols = ['distance', 'temperature', 'humidity']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [ ]:
from haversine import haversine

df['distance'] = df.apply(lambda row: haversine(
    (row['cust_lat'], row['cust_lon']),
    (row['rest_lat'], row['rest_lon'])
), axis=1)



df['order_hour'] = pd.to_datetime(df['order_time']).dt.hour
df['is_rush_hour'] = df['order_hour'].apply(lambda x: 1 if (7 <= x <=9 or 17 <= x <= 20) else 0)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




history = model.fit(train_data, epochs=20, validation_data=val_data)


In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    # Train and evaluate model on each fold
